In [2]:
import wandb
import numpy as np
from sklearn import datasets 
from  sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report, confusion_matrix
from sklearn.preprocessing import label_binarize

In [3]:
wine = datasets.load_wine()

In [4]:
wine

{'data': array([[1.423e+01, 1.710e+00, 2.430e+00, ..., 1.040e+00, 3.920e+00,
         1.065e+03],
        [1.320e+01, 1.780e+00, 2.140e+00, ..., 1.050e+00, 3.400e+00,
         1.050e+03],
        [1.316e+01, 2.360e+00, 2.670e+00, ..., 1.030e+00, 3.170e+00,
         1.185e+03],
        ...,
        [1.327e+01, 4.280e+00, 2.260e+00, ..., 5.900e-01, 1.560e+00,
         8.350e+02],
        [1.317e+01, 2.590e+00, 2.370e+00, ..., 6.000e-01, 1.620e+00,
         8.400e+02],
        [1.413e+01, 4.100e+00, 2.740e+00, ..., 6.100e-01, 1.600e+00,
         5.600e+02]]),
 'target': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [5]:
print(wine.DESCR)

.. _wine_dataset:

Wine recognition dataset
------------------------

**Data Set Characteristics:**

    :Number of Instances: 178
    :Number of Attributes: 13 numeric, predictive attributes and the class
    :Attribute Information:
 		- Alcohol
 		- Malic acid
 		- Ash
		- Alcalinity of ash  
 		- Magnesium
		- Total phenols
 		- Flavanoids
 		- Nonflavanoid phenols
 		- Proanthocyanins
		- Color intensity
 		- Hue
 		- OD280/OD315 of diluted wines
 		- Proline

    - class:
            - class_0
            - class_1
            - class_2
		
    :Summary Statistics:
    
    ============================= ==== ===== ======= =====
                                   Min   Max   Mean     SD
    ============================= ==== ===== ======= =====
    Alcohol:                      11.0  14.8    13.0   0.8
    Malic Acid:                   0.74  5.80    2.34  1.12
    Ash:                          1.36  3.23    2.36  0.27
    Alcalinity of Ash:            10.6  30.0    19.5   3.3
    Ma

In [6]:
print(f"nuestras variables son {wine.feature_names}")

nuestras variables son ['alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium', 'total_phenols', 'flavanoids', 'nonflavanoid_phenols', 'proanthocyanins', 'color_intensity', 'hue', 'od280/od315_of_diluted_wines', 'proline']


In [7]:
print(f"nuestro target es  {wine.target_names}")

nuestro target es  ['class_0' 'class_1' 'class_2']


In [9]:
# Definimos variables
X, y = wine.data, wine.target

# Definir entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X ,y , test_size=0.3, random_state=357) # Dividimos los datos en entrenamiento y prueba

# Hiperparámetros
learning_rates = [0.01, 0.1, 0.2, 0.25] # Tasa de aprendizaje
max_depths = [2,3,4,5] # Profundidad máxima de los árboles individuales
n_estimators = [50, 100, 150] # Número de árboles en el conjunto
loss_functions = ['deviance'] # Función de pérdida a optimizar; 'exponential' no es adecuado para clasificación multiclase
subsamples = [0.8, 1.0] # Fracción de muestras a usar para ajustar los árboles individuales
min_samples_splits = [2, 4] # Número mínimo de muestras necesarias para dividir un nodo interno
min_samples_leafs = [1, 2] # Número mínimo de muestras requeridas para ser una hoja

# Bucle
for lr in learning_rates:
    for max_depth in max_depths:
        for n_estimator in n_estimators:
            for loss_function in loss_functions:
                for subsample in subsamples:
                    for min_samples_split in min_samples_splits:
                        for min_samples_leaf in min_samples_leafs:
                            
                            # Iniciar wandb
                            wandb.init(project="sklearn-wine", name=f"gbm_lr{lr}_depth{max_depth}_est{n_estimator}_loss{loss_function}_subsample{subsample}_minsplit{min_samples_split}_minleaf{min_samples_leaf}")
                            
                            # Registro de hp
                            wandb.config.learning_rate = lr
                            wandb.config.max_depth = max_depth
                            wandb.config.n_estimators = n_estimator
                            wandb.config.loss_function = loss_function
                            wandb.config.subsample = subsample
                            wandb.config.min_samples_split = min_samples_split
                            wandb.config.min_samples_leaf = min_samples_leaf
                            
                            # Entrenamiento
                            clf = GradientBoostingClassifier(learning_rate=lr, max_depth=max_depth, n_estimators=n_estimator, loss=loss_function, subsample=subsample, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf, random_state=357)
                            clf.fit(X_train, y_train)
                            
                            # Predicción
                            y_pred = clf.predict(X_test)
                            y_pred_proba = clf.predict_proba(X_test)
                            
                            # Cálculo de métricas
                            accuracy = accuracy_score(y_test, y_pred)
                            f1_macro = f1_score(y_test, y_pred, average="macro")
                            y_test_bin = label_binarize(y_test, classes=np.unique(y))
                            y_pred_bin = y_pred_proba.reshape(-1, len(np.unique(y)))
                            roc_auc_macro = roc_auc_score(y_test_bin, y_pred_bin, average="macro", multi_class="ovr")
                            
                            # Registrar las métricas en wandb
                            wandb.log({"accuracy": accuracy, "f1_macro": f1_macro, "roc_auc_macro": roc_auc_macro})
                            
                            # Finalizamos el experimento
                            wandb.finish()



Problem at: C:\Users\israe\AppData\Local\Temp\ipykernel_22628\4057390197.py 26 <module>


KeyboardInterrupt: 